In [1]:
import numpy as np

x = np.asarray(
    ((0, 1, 2),
    (3, 4, 5),
    (6, 7, 8))
)
print(x)
print(x.flatten())

[[0 1 2]
 [3 4 5]
 [6 7 8]]
[0 1 2 3 4 5 6 7 8]


In [10]:

import numpy as np

import TidalPy


from TidalPy.constants import G
from TidalPy.radial_solver.matrix.fundamental_solid import fundamental_matrix_generic, fundamental_matrix_orderl2

# Model planet - 2layers
N = 100
density_array = 5000. * np.ones(N)
radius_array = np.linspace(0., 1.e6, N+1)
radius_array_reduced = radius_array[1:]
volume_array = (4. / 3.) * np.pi * (radius_array[1:]**3 - radius_array[:-1]**3)
mass_array = volume_array * density_array
planet_mass = sum(mass_array)
mass_below = np.asarray([np.sum(mass_array[:i + 1]) for i in range(N)])
gravity_array = G * mass_below / (radius_array[1:]**2)
complex_shear_array = 5.e10 * np.ones(N, dtype=np.complex128)

In [46]:
fundamental_mtx, inverse_fundamental_mtx, derivative_mtx = \
        fundamental_matrix_generic(radius_array_reduced, complex_shear_array, density_array, gravity_array, order_l=3)

In [4]:
import cython

In [5]:
%load_ext cython

In [ ]:
%%cython -a -f --compile-args=/openmp -a
# distutils: language = c
# cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True, initializedcheck=False
""" Fundamental Matrix and its inverse as defined in Sabadini, Vermeerson, & Cambiotti (2016) (Hereafter cited as SVC16)

Two versions are defined, one for a generic order-l and one for order-l=2 (for performance).

Assumptions
-----------
These matrices assume an incompressible body.

References
----------
SVC16 : Sabadini, Vermeerson, & Cambiotti (2016, DOI: 10.1007/978-94-017-7552-6)
HH14  : Henning & Hurford (2014, DOI: 10.1088/0004-637X/789/1/30)
ID    : IcyDwarf Code by Marc Neveu (https://github.com/MarcNeveu/IcyDwarf/blob/master/IcyDwarf/Thermal.h)
"""

from cython.parallel cimport prange

from libc.math cimport isfinite, isinf, isnan, copysign, \
    sqrt, fabs, signbit, exp, cos, sin, log, log1p, ldexp, atan2, frexp, ceil

from libc.math cimport NAN, INFINITY
from libc.float cimport DBL_MAX, DBL_MIN, DBL_MANT_DIG

cdef inline double complex cf_build_dblcmplx(const double a, const double b) noexcept nogil:
    cdef double complex result
    cdef double* result_dbl_ptr = <double *> &result
    result_dbl_ptr[0] = a
    result_dbl_ptr[1] = b

    return result

cdef double complex cf_cinv(double complex z) noexcept nogil:

    cdef double z_real = z.real
    cdef double z_imag = z.imag
    cdef double denom = ((z_real * z_real) + (z_imag * z_imag))
    cdef double complex inv

    # Check for extreme values
    if denom == 0.:
        inv = cf_build_dblcmplx(INFINITY, NAN)
    else:
        inv = cf_build_dblcmplx(z_real / denom, -z_imag / denom)

    return inv

from libc.math cimport pi

from TidalPy.constants cimport G
from TidalPy.utilities.math.complex cimport cf_build_dblcmplx



cdef void cf_fundamental_matrix(
        Py_ssize_t num_radial_slices,
        double* radius_array_ptr,
        double* density_array_ptr,
        double* gravity_array_ptr,
        double complex* complex_shear_array_ptr,
        double complex* fundamental_mtx_ptr,
        double complex* inverse_fundamental_mtx_ptr,
        double complex* derivative_mtx_ptr,
        unsigned char degree_l = 2,
        double G_to_use = G
        ) noexcept nogil:
    """ Construct the fundamental matrix and its inverse for a generic order-l

    See Eq. 2.42 of SVC16

    Assumptions
    -----------
    - These matrices assume an incompressible body.

    Parameters
    ----------
    num_radial_slices : Py_ssize_t
        Number of radial slices
    radius_array_ptr : double
        Pointer to array of Radius values [m]
    density_array_ptr : double
        Pointer to array of Density at each radius [kg m-3]
    gravity_array_ptr : double
        Pointer to array_ptr of acceleration due to gravity at each radius [m s-2]
    complex_shear_array : double complex
        Pointer to array of Complex shear modulus at each radius [Pa]
    fundamental_mtx_ptr : double complex*
        _Return Value_
        6 x 6 matrix of double complex values
        Fundamental matrix used in the propagation technique
    inverse_fundamental_mtx_ptr : double complex*
        _Return Value_
        The inverse of the fundamental matrix used in the propagation technique
    derivative_mtx_ptr : double complex*
        _Return Value_
        The matrix, A, that satisfies the equation dy/dr = A * y
    degree_l : unsigned char, default=2
        Harmonic degree.
    G_to_use : double, default=G
        Gravitational constant used in calculations. Can be provided for non-dimensionalized solutions.
    """

    cdef Py_ssize_t r_i, index_shift
    
    cdef double radius, gravity, density
    cdef double complex complex_shear
    
    cdef double r_inv, rl, rlp1, rlp2, rlp3, rnl, rnlm2, rlm1, rgp, piGp
    cdef double coeff, d_coeff_1, d_coeff_2, d_coeff_3, d_coeff_4, d_coeff_5, d_coeff_6
    cdef double complex mu_inv, rgp_s, r_s, pr_s

    # Degree-l Optimizations
    cdef double degree_l_dbl = <double> degree_l
    cdef double dlm1         = (2. * degree_l_dbl - 1.)
    cdef double l2p3lm1      = (degree_l_dbl**2 + 3. * degree_l_dbl - 1.)
    cdef double l2mlm3       = (degree_l_dbl**2 - degree_l_dbl - 3.)
    cdef double lp1          = (degree_l_dbl + 1.)
    cdef double lp2          = (degree_l_dbl + 2.)
    cdef double lp3          = (degree_l_dbl + 3.)
    cdef double l2m1         = (degree_l_dbl**2 - 1.)
    cdef double dlp1         = (2. * degree_l_dbl + 1.)
    cdef double dlp3         = (2. * degree_l_dbl + 3.)

    for r_i in prange(num_radial_slices):

        # Shift index by 36 (for the inner 6x6 matrix)
        index_shift = r_i * 36

        # Unpack radially dependent variables
        radius        = radius_array_ptr[r_i]
        complex_shear = complex_shear_array_ptr[r_i]
        gravity       = gravity_array_ptr[r_i]
        density       = density_array_ptr[r_i]

        # Radius-based optimizations
        r_inv  = 1. / radius
        mu_inv = cf_cinv(complex_shear)
        rl     = radius**degree_l
        rlp1   = radius**(degree_l + 1)
        rlp2   = radius**(degree_l + 2)
        rlp3   = radius**(degree_l + 3)
        rnl    = radius**(-degree_l)
        rnlm2  = radius**(-degree_l - 2)
        rlm1   = radius**(degree_l - 1)
        rgp    = radius * gravity * density
        rgp_s  = rgp * mu_inv
        r_s    = radius * mu_inv
        pr_s   = density * r_s
        piGp   = pi * G_to_use * density
        
        # D Coefficients
        coeff = (1. / dlp1)
        d_coeff_1 = coeff * lp1 / rlp1
        d_coeff_2 = coeff * degree_l_dbl * lp1 / (2. * dlm1 * rlm1)
        d_coeff_3 = coeff * 1. / rlm1
        d_coeff_4 = coeff * degree_l_dbl * rl
        d_coeff_5 = coeff * rlp2 * degree_l_dbl * lp1 / (2. * dlp3)
        d_coeff_6 = coeff * -rlp1

        
        # Build Fundamental Matrix (zeros do not need to be specifically stated as they were put in at initialization)
        #     Eq. 2.42 in SVC
        ## Row 1
        fundamental_mtx_ptr[index_shift + 0]  = degree_l_dbl * rlp1 / (2. * dlp3)
        fundamental_mtx_ptr[index_shift + 1]  = rlm1
        fundamental_mtx_ptr[index_shift + 2]  = 0.
        fundamental_mtx_ptr[index_shift + 3]  = lp1 * rnl / (2. * dlm1)
        fundamental_mtx_ptr[index_shift + 4]  = rnlm2
        fundamental_mtx_ptr[index_shift + 5]  = 0.

        ## Row 2
        fundamental_mtx_ptr[index_shift + 6]  = lp3 * rlp1 / (2. * dlp3 * lp1)
        fundamental_mtx_ptr[index_shift + 7]  = rlm1 / degree_l_dbl
        fundamental_mtx_ptr[index_shift + 8]  = 0.
        fundamental_mtx_ptr[index_shift + 9]  = (2. - degree_l_dbl) * rnl / (2. * degree_l_dbl * dlm1)
        fundamental_mtx_ptr[index_shift + 10] = -rnlm2 / lp1
        fundamental_mtx_ptr[index_shift + 11] = 0.

        ## Row 3
        # RECORD: Believe there is a typo in HH14, they have the radius^l only on one term instead of both.
        fundamental_mtx_ptr[index_shift + 12] = (degree_l_dbl * rgp + 2. * l2mlm3 * complex_shear) * rl / (2. * dlp3)
        fundamental_mtx_ptr[index_shift + 13] = (rgp + 2. * (degree_l_dbl - 1.) * complex_shear) * radius**(degree_l_dbl - 2.)
        fundamental_mtx_ptr[index_shift + 14] = -density * rl
        fundamental_mtx_ptr[index_shift + 15] = (lp1 * rgp - 2. * l2p3lm1 * complex_shear) / (2. * dlm1 * rlp1)
        fundamental_mtx_ptr[index_shift + 16] = (rgp - 2. * lp2 * complex_shear) / rlp3
        fundamental_mtx_ptr[index_shift + 17] = -density / rlp1

        ## Row 4
        fundamental_mtx_ptr[index_shift + 18] = degree_l_dbl * lp2 * complex_shear * rl / (dlp3 * lp1)
        fundamental_mtx_ptr[index_shift + 19] = 2 * (degree_l_dbl - 1.) * complex_shear * radius**(degree_l_dbl - 2.) / degree_l_dbl
        fundamental_mtx_ptr[index_shift + 20] = 0.
        fundamental_mtx_ptr[index_shift + 21] = l2m1 * complex_shear / (degree_l_dbl * dlm1 * rlp1)
        fundamental_mtx_ptr[index_shift + 22] = 2. * lp2 * complex_shear / (lp1 * rlp3)
        fundamental_mtx_ptr[index_shift + 23] = 0.

        ## Row 5
        fundamental_mtx_ptr[index_shift + 24] = 0.
        fundamental_mtx_ptr[index_shift + 25] = 0.
        fundamental_mtx_ptr[index_shift + 26] = -rl
        fundamental_mtx_ptr[index_shift + 27] = 0.
        fundamental_mtx_ptr[index_shift + 28] = 0.
        fundamental_mtx_ptr[index_shift + 29] = -1. / rlp1

        ## Row 6
        fundamental_mtx_ptr[index_shift + 30] = 2. * piGp * degree_l_dbl * rlp1 / dlp3
        fundamental_mtx_ptr[index_shift + 31] = 4. * piGp * rlm1
        fundamental_mtx_ptr[index_shift + 32] = -dlp1 * rlm1
        fundamental_mtx_ptr[index_shift + 33] = 2 * piGp * lp1 / (dlm1 * rl)
        fundamental_mtx_ptr[index_shift + 34] = 4. * piGp / rlp2
        fundamental_mtx_ptr[index_shift + 35] = 0.

        # Inverse of the Fundamental Matrix
        # This function manually defines the inverse matrix which is about 15--30% faster than a version that uses
        #    np.linalg.inv() to calculate the inverse matrix.
        #
        # From SVC16 Eq. 2.45: Fundamental Inverse = D_Mtx * Y^Bar_Mtx
        # D_Mtx is a diagonal matrix with
        # 1/(2l+1) * [ ... ]
        # We are going to multiple first and just write down the fundamental matrix inverse to avoid the additional D*Ybar
        #    calculation.

        ## Row 1
        inverse_fundamental_mtx_ptr[index_shift + 0]  = d_coeff_1 * (rgp_s - 2. * lp2)
        inverse_fundamental_mtx_ptr[index_shift + 1]  = d_coeff_1 * (2. * degree_l_dbl * lp2)
        inverse_fundamental_mtx_ptr[index_shift + 2]  = d_coeff_1 * (-r_s)
        inverse_fundamental_mtx_ptr[index_shift + 3]  = d_coeff_1 * (degree_l_dbl * r_s)
        inverse_fundamental_mtx_ptr[index_shift + 4]  = d_coeff_1 * (pr_s)
        inverse_fundamental_mtx_ptr[index_shift + 5]  = 0.

        ## Row 2
        inverse_fundamental_mtx_ptr[index_shift + 6]  = d_coeff_2 * (-rgp_s + 2. * l2p3lm1 / lp1)
        inverse_fundamental_mtx_ptr[index_shift + 7]  = d_coeff_2 * (-2. * l2m1)
        inverse_fundamental_mtx_ptr[index_shift + 8]  = d_coeff_2 * (r_s)
        inverse_fundamental_mtx_ptr[index_shift + 9]  = d_coeff_2 * ((2. - degree_l_dbl) * r_s)
        inverse_fundamental_mtx_ptr[index_shift + 10] = d_coeff_2 * (-pr_s)
        inverse_fundamental_mtx_ptr[index_shift + 11] = 0.

        ## Row 3
        inverse_fundamental_mtx_ptr[index_shift + 12] = d_coeff_3 * (4. * piGp)
        inverse_fundamental_mtx_ptr[index_shift + 13] = 0.
        inverse_fundamental_mtx_ptr[index_shift + 14] = 0.
        inverse_fundamental_mtx_ptr[index_shift + 15] = 0.
        inverse_fundamental_mtx_ptr[index_shift + 16] = 0.
        inverse_fundamental_mtx_ptr[index_shift + 17] = -d_coeff_3

        ## Row 4
        inverse_fundamental_mtx_ptr[index_shift + 18] = d_coeff_4 * (rgp_s + 2. * (degree_l_dbl - 1.))
        inverse_fundamental_mtx_ptr[index_shift + 19] = d_coeff_4 * (2. * l2m1)
        inverse_fundamental_mtx_ptr[index_shift + 20] = d_coeff_4 * (-r_s)
        inverse_fundamental_mtx_ptr[index_shift + 21] = d_coeff_4 * (-lp1 * r_s)
        inverse_fundamental_mtx_ptr[index_shift + 22] = d_coeff_4 * (pr_s)
        inverse_fundamental_mtx_ptr[index_shift + 23] = 0.

        ## Row 5
        inverse_fundamental_mtx_ptr[index_shift + 24] = d_coeff_5 * (-rgp_s - 2. * l2mlm3 / degree_l_dbl)
        inverse_fundamental_mtx_ptr[index_shift + 25] = d_coeff_5 * (-2. * degree_l_dbl * lp2)
        inverse_fundamental_mtx_ptr[index_shift + 26] = d_coeff_5 * (r_s)
        inverse_fundamental_mtx_ptr[index_shift + 27] = d_coeff_5 * (lp3 * r_s)
        inverse_fundamental_mtx_ptr[index_shift + 28] = d_coeff_5 * (-pr_s)
        inverse_fundamental_mtx_ptr[index_shift + 29] = 0.

        ## Row 6
        inverse_fundamental_mtx_ptr[index_shift + 30] = d_coeff_6 * (4. * piGp * radius)
        inverse_fundamental_mtx_ptr[index_shift + 31] = 0.
        inverse_fundamental_mtx_ptr[index_shift + 32] = 0.
        inverse_fundamental_mtx_ptr[index_shift + 33] = 0.
        inverse_fundamental_mtx_ptr[index_shift + 34] = d_coeff_6 * dlp1
        inverse_fundamental_mtx_ptr[index_shift + 35] = d_coeff_6 * (-radius)

        # Build derivative matrix
        # Defined in SV04 -- Only valid for the incompressible case.
        # See SVC16 Eq. 1.95
        #    Note: the lambda in SVC16 is defined as bulk_mod - (2. / 3.) * shear (Eq. 1.77; 2nd Lame parameter),
        #    for the incompressible assumption we will assume the ratio that SVC16 use (lambda / beta) -> 1 as K -> inf
        #    See SVC16 Eq. 1.95 for a compressible version. Take limit as K->inf to find below.
        ## Row 1
        derivative_mtx_ptr[index_shift + 0]  = -2. * r_inv
        derivative_mtx_ptr[index_shift + 1]  = degree_l_dbl * lp1 * r_inv
        derivative_mtx_ptr[index_shift + 2]  = 0.
        derivative_mtx_ptr[index_shift + 3]  = 0.
        derivative_mtx_ptr[index_shift + 4]  = 0.
        derivative_mtx_ptr[index_shift + 5]  = 0.

        ## Row 2
        derivative_mtx_ptr[index_shift + 6]  = -1. * r_inv
        derivative_mtx_ptr[index_shift + 7]  = r_inv
        derivative_mtx_ptr[index_shift + 8]  = 0.
        derivative_mtx_ptr[index_shift + 9]  = mu_inv
        derivative_mtx_ptr[index_shift + 10] = 0.
        derivative_mtx_ptr[index_shift + 11] = 0.

        ## Row 3
        derivative_mtx_ptr[index_shift + 12] = (4. * r_inv) * (3. * complex_shear * r_inv - density * gravity)
        derivative_mtx_ptr[index_shift + 13] = -degree_l_dbl * lp1 * r_inv * (6. * complex_shear * r_inv - density * gravity)
        derivative_mtx_ptr[index_shift + 14] = 0.
        derivative_mtx_ptr[index_shift + 15] = degree_l_dbl * lp1 * r_inv
        derivative_mtx_ptr[index_shift + 16] = -density * lp1 * r_inv
        derivative_mtx_ptr[index_shift + 17] = density

        ## Row 4
        derivative_mtx_ptr[index_shift + 18] = (-1. * r_inv) * (6. * complex_shear * r_inv - density * gravity)
        derivative_mtx_ptr[index_shift + 19] = 2. * (2. * degree_l_dbl**2 + 2. * degree_l_dbl - 1.) * complex_shear * (r_inv**2)
        derivative_mtx_ptr[index_shift + 20] = -r_inv
        derivative_mtx_ptr[index_shift + 21] = -3. * r_inv
        derivative_mtx_ptr[index_shift + 22] = density * r_inv
        derivative_mtx_ptr[index_shift + 23] = 0.

        ## Row 5
        derivative_mtx_ptr[index_shift + 24] = -4. * piGp
        derivative_mtx_ptr[index_shift + 25] = 0.
        derivative_mtx_ptr[index_shift + 26] = 0.
        derivative_mtx_ptr[index_shift + 27] = 0.
        derivative_mtx_ptr[index_shift + 28] = -lp1 * r_inv
        derivative_mtx_ptr[index_shift + 29] = 1.

        ## Row 6
        derivative_mtx_ptr[index_shift + 30] = -4. * piGp * (degree_l_dbl + 1) * r_inv
        derivative_mtx_ptr[index_shift + 31] = 4. * piGp * degree_l_dbl * lp1 * r_inv
        derivative_mtx_ptr[index_shift + 32] = 0.
        derivative_mtx_ptr[index_shift + 33] = 0.
        derivative_mtx_ptr[index_shift + 34] = 0.
        derivative_mtx_ptr[index_shift + 35] = (degree_l_dbl - 1.) * r_inv


def fundamental_matrix(
        double[::1] radius_array_view,
        double[::1] density_array_view,
        double[::1] gravity_array_view,
        double complex[::1] complex_shear_array_view,
        double complex[:, :, ::1] fundamental_mtx_view,
        double complex[:, :, ::1] fundamental_mtx_inverse_view,
        double complex[:, :, ::1] derivative_mtx_view,  
        unsigned char degree_l = 2,
        double G_to_use = G
        ):
    """ Construct the fundamental matrix and its inverse using harmonic degree l.

    See Eq. 2.42 of SVC16

    Assumptions
    -----------
    - These matrices assume an incompressible body.

    Parameters
    ----------
    radius_array_view : double
        Pointer to array of Radius values [m]
    density_array_view : double
        Pointer to array of Density at each radius [kg m-3]
    gravity_array_view : double
        Pointer to array of acceleration due to gravity at each radius [m s-2]
    complex_shear_array_view : double complex
        Pointer to array of Complex shear modulus at each radius [Pa]
    fundamental_mtx_view : double complex[:, ::1]
        _Return Value_
        Fundamental matrix used in the propagation technique
    fundamental_mtx_inverse_view : double complex[:, ::1]
        _Return Value_
        The inverse of the fundamental matrix used in the propagation technique
    derivative_mtx_view : double complex[:, ::1]
        _Return Value_
        The matrix, A, that satisfies the equation dy/dr = A * y
    degree_l : unsigned char, default=2
        Harmonic degree.
    G_to_use : double, default=G
        Gravitational constant used in calculations. Can be provided for non-dimensionalized solutions.
    """ 

    cdef Py_ssize_t num_radial_slices
    num_radial_slices = len(radius_array_view)

    # Check for unexpected shapes and sizes of return matrices
    if len(density_array_view) != num_radial_slices:
        raise ValueError('Unexpected size encountered for density array.')
    if len(gravity_array_view) != num_radial_slices:
        raise ValueError('Unexpected size encountered for gravity array.')
    if len(complex_shear_array_view) != num_radial_slices:
        raise ValueError('Unexpected size encountered for complex shear array.')
    if (fundamental_mtx_view.shape[0] != num_radial_slices) and \
        (fundamental_mtx_view.shape[1] != 6) and \
        (fundamental_mtx_view.shape[2] != 6):
        raise ValueError('Unexpected shape encountered for Fundamental Matrix.')
    if (fundamental_mtx_inverse_view.shape[0] != num_radial_slices) and \
        (fundamental_mtx_inverse_view.shape[1] != 6) and \
        (fundamental_mtx_inverse_view.shape[2] != 6):
        raise ValueError('Unexpected shape encountered for Fundamental Matrix (inv).')
    if (derivative_mtx_view.shape[0] != num_radial_slices) and \
        (derivative_mtx_view.shape[1] != 6) and \
        (derivative_mtx_view.shape[2] != 6):
        raise ValueError('Unexpected shape encountered for Derivative Matrix.')

    # Call cdef function.
    cf_fundamental_matrix(
        num_radial_slices,
        &radius_array_view[0],
        &density_array_view[0],
        &gravity_array_view[0],
        &complex_shear_array_view[0],
        &fundamental_mtx_view[0, 0, 0],
        &fundamental_mtx_inverse_view[0, 0, 0],
        &derivative_mtx_view[0, 0, 0],
        degree_l,
        G_to_use
    )



Content of stdout:
_cython_magic_f28cf44a80e548129d3c9d4579d95aa346471c7d.c
C:\Users\jrenaud\.ipython\cython\_cython_magic_f28cf44a80e548129d3c9d4579d95aa346471c7d.c(1492): warning C4005: '__pyx_nonatomic_int_type': macro redefinition
C:\Users\jrenaud\.ipython\cython\_cython_magic_f28cf44a80e548129d3c9d4579d95aa346471c7d.c(1443): note: see previous definition of '__pyx_nonatomic_int_type'
   Creating library C:\Users\jrenaud\.ipython\cython\Users\jrenaud\.ipython\cython\_cython_magic_f28cf44a80e548129d3c9d4579d95aa346471c7d.cp311-win_amd64.lib and object C:\Users\jrenaud\.ipython\cython\Users\jrenaud\.ipython\cython\_cython_magic_f28cf44a80e548129d3c9d4579d95aa346471c7d.cp311-win_amd64.exp
Generating code
Finished generating code

In [81]:
# fundamental_mtx_cy = np.empty((6, 6, N), dtype=np.complex128, order='C')
fundamental_mtx_cy = np.arange(6*6*N).reshape(N, 6, 6)
fundamental_mtx_cy[0, :,:].T

array([[ 0,  6, 12, 18, 24, 30],
       [ 1,  7, 13, 19, 25, 31],
       [ 2,  8, 14, 20, 26, 32],
       [ 3,  9, 15, 21, 27, 33],
       [ 4, 10, 16, 22, 28, 34],
       [ 5, 11, 17, 23, 29, 35]])

In [47]:
fundamental_mtx_cy = np.empty((density_array.size, 6, 6), dtype=np.complex128, order='C')
inverse_fundamental_mtx_cy = np.empty((density_array.size, 6, 6), dtype=np.complex128, order='C')
derivative_mtx_cy = np.empty((density_array.size, 6, 6), dtype=np.complex128, order='C')

fundamental_matrix(
    radius_array_reduced,
    density_array,
    gravity_array,
    complex_shear_array,
    fundamental_mtx_cy,
    inverse_fundamental_mtx_cy,
    derivative_mtx_cy,
    degree_l = 3)


for i in range(N):
    print('\n', i, end=':  ')
    print(np.allclose(fundamental_mtx_cy[i, :, :], fundamental_mtx[:, :, i]), end='  ')
    print(np.allclose(inverse_fundamental_mtx_cy[i, :, :], inverse_fundamental_mtx[:, :, i]), end='  ')
    print(np.allclose(derivative_mtx_cy[i, :, :], derivative_mtx[:, :, i]), end='  ')
    
    
np.save('solid_fundamental_matrix_l3.npy', fundamental_mtx_cy)
np.save('solid_inverse_fundamental_matrix_l3.npy', inverse_fundamental_mtx_cy)
np.save('solid_derivative_matrix_l3.npy', derivative_mtx_cy)


 0:  True  True  True  
 1:  True  True  True  
 2:  True  True  True  
 3:  True  True  True  
 4:  True  True  True  
 5:  True  True  True  
 6:  True  True  True  
 7:  True  True  True  
 8:  True  True  True  
 9:  True  True  True  
 10:  True  True  True  
 11:  True  True  True  
 12:  True  True  True  
 13:  True  True  True  
 14:  True  True  True  
 15:  True  True  True  
 16:  True  True  True  
 17:  True  True  True  
 18:  True  True  True  
 19:  True  True  True  
 20:  True  True  True  
 21:  True  True  True  
 22:  True  True  True  
 23:  True  True  True  
 24:  True  True  True  
 25:  True  True  True  
 26:  True  True  True  
 27:  True  True  True  
 28:  True  True  True  
 29:  True  True  True  
 30:  True  True  True  
 31:  True  True  True  
 32:  True  True  True  
 33:  True  True  True  
 34:  True  True  True  
 35:  True  True  True  
 36:  True  True  True  
 37:  True  True  True  
 38:  True  True  True  
 39:  True  True  True  
 40:  Tru

In [31]:

rar = radius_array_reduced
csr = complex_shear_array
drr = density_array
grr = gravity_array

fundamental_mtx, inverse_fundamental_mtx, derivative_mtx = \
        fundamental_matrix_generic(rar, csr, drr, grr, order_l=2)


In [32]:
# 16us 13.1us 13us
# Array Version
# 27.3us, 27.5us, 27.3us, 27.1us


%timeit fundamental_matrix_generic(rar, csr, drr, grr, order_l=2)

27.1 µs ± 139 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [44]:
# 906us 921us 906us
# 2.16us, 2.14us, 2.16us
# 2.14us, 2.1us, 2.07us

# Array Version
# 26.7us, 26.4us, 27us
# 26.6us, 27.1us, 26.4us
# 7.94us, 7.89us


A = 100


# 5: 5.53us
# 10: 5.95us
# 20: 6.4us

fundamental_mtx_cy = np.empty((density_array.size, 6, 6), dtype=np.complex128, order='C')
inverse_fundamental_mtx_cy = np.empty((density_array.size, 6, 6), dtype=np.complex128, order='C')
derivative_mtx_cy = np.empty((density_array.size, 6, 6), dtype=np.complex128, order='C')

%timeit fundamental_matrix(rar, drr, grr, csr, fundamental_mtx_cy, inverse_fundamental_mtx_cy, derivative_mtx_cy, degree_l = 2)

7.76 µs ± 119 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
